In [1]:
import dgl
import dgl.data
import dgl.nn as gnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Using backend: pytorch


# Обучение графовых нейросетей на GPU

In [2]:
# Машина должна быть настроена должным образом
assert torch.cuda.is_available()

In [3]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [4]:
class GCN(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.conv1 = gnn.GraphConv(n_input, n_hidden)
        self.conv2 = gnn.GraphConv(n_hidden, n_output)
    
    def forward(self, G, in_features):
        out = F.relu(self.conv1(G, in_features))
        out = self.conv2(G, out)
        return out

In [5]:
def train(model, g):
    optimizer = optim.Adam(model.parameters(), lr=.01)
    criterion = nn.CrossEntropyLoss()

    best_val_acc, best_test_acc = 0, 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    n_epochs = 100
    for epoch in range(n_epochs):
        logits = model(g, features)
        loss = criterion(logits[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        with torch.no_grad():
            predictions = logits.argmax(dim=1)
            train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
            val_acc = (predictions[val_mask] == labels[val_mask]).float().mean()
            test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

            if best_val_acc < val_acc:
                best_val_acc = val_acc
                best_test_acc = test_acc

        if not epoch % 5:
            print(f'In epoch {epoch}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})')

In [6]:
n_input = g.ndata['feat'].shape[1]
n_hidden = 16
n_out = dataset.num_classes

In [7]:
%%time
model = GCN(n_input, n_hidden, n_out)
train(model, g)

In epoch 0, loss: 1.946, val acc: 0.238 (best 0.238), test acc: 0.255 (best 0.255)
In epoch 5, loss: 1.884, val acc: 0.564 (best 0.564), test acc: 0.607 (best 0.607)
In epoch 10, loss: 1.796, val acc: 0.638 (best 0.638), test acc: 0.692 (best 0.692)
In epoch 15, loss: 1.685, val acc: 0.694 (best 0.694), test acc: 0.710 (best 0.710)
In epoch 20, loss: 1.551, val acc: 0.722 (best 0.722), test acc: 0.723 (best 0.723)
In epoch 25, loss: 1.396, val acc: 0.746 (best 0.746), test acc: 0.724 (best 0.724)
In epoch 30, loss: 1.227, val acc: 0.752 (best 0.752), test acc: 0.728 (best 0.728)
In epoch 35, loss: 1.051, val acc: 0.754 (best 0.754), test acc: 0.729 (best 0.727)
In epoch 40, loss: 0.880, val acc: 0.754 (best 0.754), test acc: 0.734 (best 0.727)
In epoch 45, loss: 0.722, val acc: 0.758 (best 0.758), test acc: 0.736 (best 0.735)
In epoch 50, loss: 0.584, val acc: 0.760 (best 0.764), test acc: 0.743 (best 0.742)
In epoch 55, loss: 0.469, val acc: 0.772 (best 0.772), test acc: 0.745 (best 0

In [9]:
%%time
# Для обучении на GPU помещаем модель и граф на GPU.
model = GCN(n_input, n_hidden, n_out).to('cuda')
train(model, g.to('cuda'))

In epoch 0, loss: 1.945, val acc: 0.196 (best 0.196), test acc: 0.214 (best 0.214)
In epoch 5, loss: 1.880, val acc: 0.476 (best 0.476), test acc: 0.498 (best 0.498)
In epoch 10, loss: 1.791, val acc: 0.628 (best 0.630), test acc: 0.660 (best 0.640)
In epoch 15, loss: 1.681, val acc: 0.652 (best 0.656), test acc: 0.668 (best 0.667)
In epoch 20, loss: 1.551, val acc: 0.658 (best 0.658), test acc: 0.655 (best 0.655)
In epoch 25, loss: 1.403, val acc: 0.652 (best 0.658), test acc: 0.662 (best 0.655)
In epoch 30, loss: 1.241, val acc: 0.672 (best 0.672), test acc: 0.687 (best 0.687)
In epoch 35, loss: 1.074, val acc: 0.702 (best 0.702), test acc: 0.713 (best 0.713)
In epoch 40, loss: 0.910, val acc: 0.718 (best 0.718), test acc: 0.736 (best 0.732)
In epoch 45, loss: 0.756, val acc: 0.726 (best 0.726), test acc: 0.746 (best 0.746)
In epoch 50, loss: 0.620, val acc: 0.746 (best 0.746), test acc: 0.756 (best 0.756)
In epoch 55, loss: 0.504, val acc: 0.760 (best 0.760), test acc: 0.764 (best 0